# Import Libraries

In [2]:
import re
import time
import os

In [3]:
import numpy as np
import cv2

In [4]:
from PIL import Image

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
import pytesseract
import easyocr

# Paths

In [8]:
TESSERACT_PATH    = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
CHROMEDRIVER_PATH = r"C:\Program Files\Google\chromedriver-win64\chromedriver.exe"
CHROME_PATH       = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

In [9]:
login_url = r"https://goaps.iisc.ac.in/login"

In [10]:
credentials_file = r"credential.txt"

In [11]:
pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH

In [12]:
'''
easyocr.Reader is a class provided by the EasyOCR library, 
which is used for Optical Character Recognition (OCR). 

It allows the script to extract text from images, 
such as the arithmetic CAPTCHA in your code.
'''

# ['en']: This specifies the language(s) for OCR.
# In this case, ['en'] indicates that the Reader will recognize English text.

# gpu=False: This disables GPU acceleration for EasyOCR. 
# When set to False, the OCR processing runs on the CPU instead of a GPU.

reader = easyocr.Reader(['en'], gpu=False)

Using CPU. Note: This module is much faster with a GPU.


# Code Functionality

<h3><center> preprocess_image() </center></h3>

The `preprocess_image` function is designed to process an input image (likely a CAPTCHA image) to make it easier for optical character recognition (OCR) tools like EasyOCR or Tesseract to accurately extract text.

In [15]:
def preprocess_image(path):

    # Read the image in grayscale
    # Grayscale simplifies processing by reducing color complexity, which is often unnecessary for text extraction in CAPTCHAs.
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)



    
    # Apply denoising to reduce noise
    '''
       cv2.fastNlMeansDenoising : is a non-local means denoising algorithm that smooths the image while preserving edges.
    '''
    img = cv2.fastNlMeansDenoising(img, None, 30, 7, 21)




    
    # Apply binary thresholding 
    # Converts the grayscale image to a binary image (black and white) to enhance contrast between text and background.
    '''
        cv2.THRESH_BINARY_INV : Inverts the binary threshold, making the text white (255) and the background black (0). 
                                This is useful for CAPTCHAs where text is often lighter than the background.

        _ : captures the threshold value returned by the function, which is not used here.
    '''
    _, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY_INV)





    
    # Create a rectangular kernel for morphological operations
    # Defines a 2x2 rectangular structuring element (kernel) for morphological operations.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))



    

    # Apply morphological closing
    # Removes small holes or gaps in the binary image (e.g., within letters) to make text more continuous and readable.
    '''
        cv2.morphologyEx : performs a morphological operation.

        cv2.MORPH_CLOSE : Applies a closing operation (dilation followed by erosion).
                          Dilation expands white regions (text) to fill small gaps.
                          Erosion shrinks them back to remove noise introduced by dilation.
    '''
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)




    
    # Resize the image
    # Enlarges the image to improve OCR accuracy.
    # Increasing the image size makes small or fine details (like thin text) more distinguishable for OCR tools.
    resized = cv2.resize(cleaned, (cleaned.shape[1]*3, cleaned.shape[0]*3))

    
    return resized

<h3><center> clean_expression() </center></h3>

* The `clean_expression` function is designed to process raw text output from an OCR (Optical Character Recognition) system, which extracts text from CAPTCHA images.

* CAPTCHAs often contain arithmetic expressions (e.g., "2+3*4"), but OCR can misinterpret characters due to image noise or font variations (e.g., mistaking 'O' for '0' or 'I' for '1').

* This function cleans the text to ensure it’s a valid arithmetic expression that can be evaluated.

In [17]:
'''
Input: The function takes a single parameter, text, which is the raw string extracted by the OCR system 
       (e.g., "2+O*4" or "S+3").
'''

def clean_expression(text):
    
    print(f"Raw OCR text before clean: '{text}'")


    
    # dictionary that maps common OCR errors to their correct characters or empty strings
    replace_map = {
                   'O':'0' , 'o':'0' , 'D':'0' , 'I':'1' , 'l':'1' , 
                   'Z':'2' , 'S':'5' , 'B':'8' , '|':'1' , '?':''  , 
                   ':':''  , ' ':''  , '\n':'' , '\r':''
                  }



    
    
    # Iterates through the replace_map dictionary and replaces each key (k) in the text with its corresponding value (v).
    '''
        For example, if text = "2+O*4", 
        this loop converts it to 2+0*4 by replacing 'O' with '0'.
    '''
    for k, v in replace_map.items():
        text = text.replace(k, v)





    
    '''
        Uses the re.sub function from the re (regular expression) module to remove any characters that are not digits (0-9), 
        or the operators +, -, *, /.

        The pattern [^0-9+\-*/] matches any character that is not a digit or one of the allowed operators.
        These characters are replaced with an empty string (""), effectively removing them.
    '''
    text = re.sub(r"[^0-9+\-*/]", "", text)



    

    
    '''
        Strip Operators: Removes any leading or trailing +, -, *, or / characters from the string using the strip method.
        This ensures the expression doesn’t start or end with operators, which could cause errors when evaluating the expression.

        Example: If text = "+2+0*4-", this step results in 2+0*4.
    '''
    text = text.strip("+-*/")
    print(f" Cleaned expression: '{text}'")


    
    return text

<>:36: SyntaxWarning: invalid escape sequence '\-'
<>:36: SyntaxWarning: invalid escape sequence '\-'
C:\Users\HP\AppData\Local\Temp\ipykernel_9328\226031953.py:36: SyntaxWarning: invalid escape sequence '\-'
  '''


<h3><center> evaluate_expression() </center></h3>

* The function `evaluate_expression` takes a string expr (e.g., a mathematical expression like "2+3*4") as input and attempts to evaluate it to compute the result.

* This is likely used to solve an arithmetic CAPTCHA by calculating the answer to the expression extracted from an image.

In [19]:
def evaluate_expression(expr):
    
    try:
        result = eval(expr)
        print(f"Solved CAPTCHA: {expr} = {result}")
        return str(int(result))
        
    except Exception as e:
        print("Evaluation failed:", e)
        return None

<h3><center> main() </center></h3>

In [21]:
def main():

    '''
       Initialization of WebDriver
    '''
    # Creates a service object for ChromeDriver, which is the executable that controls Chrome
    service = Service(CHROMEDRIVER_PATH)
    
    # Sets up configuration options for the Chrome browser.
    options = webdriver.ChromeOptions()

    # Opens the browser in maximized mode for better visibility and interaction.
    options.add_argument("--start-maximized")

    # Initializes the Chrome WebDriver with the specified service and options, creating a controllable browser instance.
    driver = webdriver.Chrome(service=service, options=options)








    
    
    '''
       reading a file containing login credentials (usernames and passwords) and 
       processing them into a list of lists.
    '''
    with open(credentials_file, "r") as f:
        credentials = [
                       line.strip().split(",") 
                       for line in f 
                       if line.strip()
                      ]






    
    '''
        solve captcha
    '''
    for username, password in credentials :
        print(f"\n Trying login for user: {username}")
    
        try:
            # driver.get("about:blank"): Navigates the browser to a blank page to ensure a clean state before loading the login page.
            driver.get("about:blank")
            time.sleep(1)
            driver.get(login_url)
    
    
            
            
            '''
               Fills in the login form with the username and password.
            '''
            # Waits up to 10 seconds for the username input field to appear on the page.
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
           
            # Locates the username input field and enters the username.
            driver.find_element(By.ID, "username").send_keys(username)
            
            # Locates the password input field and enters the password.
            driver.find_element(By.ID, "password").send_keys(password)
    
    
    
    
            
    
    
    
            '''
                Captures the CAPTCHA image from the login page.
            '''
            captcha_img = WebDriverWait(driver, 10).until(
                                                           # Waits up to 10 seconds for an <img> element whose src attribute contains the word "captcha" (indicating it’s the CAPTCHA image). 
                                                           EC.presence_of_element_located((By.XPATH, "//img[contains(@src, 'captcha')]"))
                                                         )
            
            # Forces the browser to reload the CAPTCHA image by resetting its src attribute. This is likely to ensure the image is fresh or to trigger a redraw.
            driver.execute_script("arguments[0].src = arguments[0].src;", captcha_img)
            
            # Waits 2 seconds to ensure the CAPTCHA image is fully loaded.
            time.sleep(2)
            
            # Saves a screenshot of the CAPTCHA image to a file named captcha.png.
            captcha_img.screenshot("captcha.png")
    
    
    
    
    
    
    
    
            '''
                 Processes the CAPTCHA image to improve OCR accuracy.
            '''
            processed = preprocess_image("captcha.png")
            cv2.imwrite("processed_captcha.png", processed)
    
    
    
    
    
    
    
            
            # OCR using EasyOCR
            '''
                Uses EasyOCR to extract text from the processed CAPTCHA image.
            '''
    
            # Runs EasyOCR on the processed image to extract text. 
            # The detail=0 parameter means only the text is returned (not bounding boxes or confidence scores).
            result = reader.readtext(processed, detail=0)
            
            print(" EasyOCR result:", result)
    
            # Combines the OCR results (which may be a list of strings) into a single string and removes leading/trailing whitespace.
            ocr_text = "".join(result).strip()
    
    
    
    
    
    
    
            '''
                Cleans the OCR-extracted text and evaluates the arithmetic expression.
            '''
            cleaned_expr = clean_expression(ocr_text)
            answer = evaluate_expression(cleaned_expr)
    
    
    
    
    
    
    
            
            # Fallback using Tesseract
            '''
                Uses Tesseract OCR as a fallback if EasyOCR fails to produce a valid answer.
            '''
            # Checks if the previous attempt (EasyOCR) failed to produce a valid result.
            if not answer:
                config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789+-*/'
                
                # Converts the OpenCV image to a PIL image for Tesseract.
                img_pil = Image.fromarray(processed)
                
                # Runs Tesseract to extract text from the image.
                tesseract_text = pytesseract.image_to_string(img_pil, config=config)
                
                # Prints the Tesseract result for debugging.
                print(" Tesseract result:", tesseract_text)
                
                # Cleans the Tesseract-extracted text.
                cleaned_expr = clean_expression(tesseract_text)
                
                # Evaluates the cleaned expression to get the CAPTCHA answer.
                answer = evaluate_expression(cleaned_expr)
    
            
            '''
                Provides a default CAPTCHA answer if both EasyOCR and Tesseract fail.
            '''
            if not answer:
                answer = "1234"
                print("Using fallback answer.")
        
        
        

            
            

            '''
                Submits the CAPTCHA answer and clicks the login button.
            '''
            driver.find_element(By.ID, "captcha").send_keys(answer)
            driver.find_element(By.XPATH, "//button[contains(., 'Login')]").click()



            
            
            '''
                Waits for the login attempt to complete, checking for success or failure.
            '''
            WebDriverWait(driver, 10).until(
                EC.or_(
                    EC.url_changes(login_url),
                    EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Error')]"))
                )
            )




            
    
            '''
                Checks if the login was successful and decides whether to continue or stop.
            '''
            if "login" not in driver.current_url.lower():
                print("Login successful!")
                break
            else:
                print("Login failed.")


        
        except Exception as e:
            print("Exception occurred:", e)
            driver.save_screenshot(f"error_{username}.png")
            with open(f"error_{username}.html", "w", encoding="utf-8") as f:
                f.write(driver.page_source)




    
    '''
        Keep the browser open until manually closed.
    '''
    print("\n Press CTRL+C to quit.")
    try:
        while True:
            time.sleep(15)
    except KeyboardInterrupt:
        driver.quit()
        
     

In [22]:
if __name__ == "__main__":
    main()


 Trying login for user: pandohkapish30@gmail.com 
 EasyOCR result: ['9 - 5 =']
Raw OCR text before clean: '9 - 5 ='
 Cleaned expression: '9-5'
Solved CAPTCHA: 9-5 = 4
Exception occurred: module 'selenium.webdriver.support.expected_conditions' has no attribute 'or_'

 Trying login for user: sureshraina@gmail.com 
 EasyOCR result: ['9 - & =']
Raw OCR text before clean: '9 - & ='
 Cleaned expression: '9'
Solved CAPTCHA: 9 = 9
Exception occurred: module 'selenium.webdriver.support.expected_conditions' has no attribute 'or_'

 Press CTRL+C to quit.
